### 对收集的数据的收集和简单处理完毕，接下来进行进一步处理和分析  
（收集和简单处理的代码，详见pa_wxx.ipynb）

In [2]:
"""
后续需将以下数据说明仿照kaggle: https://www.kaggle.com/datasets

key: 观测站点的唯一标识符
class: 观测的类别
expire_time_gmt: 观测的过期时间
obs_id: 观测的ID
obs_name: 观测站点的名称
valid_time_gmt: 观测的有效时间
day_ind: 一天中的时间段（白天或晚上）
temp: 温度（华氏度）
wx_icon: 天气图标的代码
icon_extd: 天气图标的扩展代码
wx_phrase: 天气短语描述
pressure_tend: 气压变化趋势 (0-稳定, 1-上升, 2-下降)
pressure_desc: 气压描述 (其实就是前一字段的描述)
dewPt: 露点温度（华氏度）
heat_index: 酷热指数
rh: 相对湿度
pressure: 气压
vis: 能见度（英里）
wc: 风寒指数
wdir: 风向（角度） —— —— 以北, 顺时针的角度(取值范围10-360), 360代表北 
wdir_cardinal: 风向的助记符号
gust: 风速瞬时值（英里/小时）
wspd: 风速（英里/小时）
max_temp: 最高温度（华氏度）
min_temp: 最低温度（华氏度）
precip_total: 总降水量（英寸）
precip_hrly: 每小时降水量（英寸）
snow_hrly: 每小时降雪量（英寸）
uv_desc: 紫外线指数描述
feels_like: 体感温度（华氏度）
uv_index: 紫外线指数 https://www.epa.gov/sunsafety/calculating-uv-index-0

(以下列除了clds，都无数据，只是爬取时有该列，但是并没有数据)
qualifier: 气象数据的限定条件
qualifier_svrty: 限定条件的严重程度
blunt_phrase: 概括的天气短语描述
terse_phrase: 简短的天气短语描述
clds: 云量 https://www.eoas.ubc.ca/courses/atsc113/flying/met_concepts/01-met_concepts/01c-cloud_coverage/index.html
water_temp: 水温
primary_wave_period: 主波周期
primary_wave_height: 主波高度
primary_swell_period: 主涌浪周期
primary_swell_height: 主涌浪高度
primary_swell_direction: 主涌浪方向
secondary_swell_period: 次涌浪周期
secondary_swell_height: 次涌浪高度
secondary_swell_direction: 次涌浪方向"""

'\nkey: 观测站点的唯一标识符\nclass: 观测的类别\nexpire_time_gmt: 观测的过期时间\nobs_id: 观测的ID\nobs_name: 观测站点的名称\nvalid_time_gmt: 观测的有效时间\nday_ind: 一天中的时间段（白天或晚上）\ntemp: 温度（华氏度）\nwx_icon: 天气图标的代码\nicon_extd: 天气图标的扩展代码\nwx_phrase: 天气短语描述\npressure_tend: 气压变化趋势\npressure_desc: 气压描述\ndewPt: 露点温度（华氏度）\nheat_index: 酷热指数\nrh: 相对湿度\npressure: 气压\nvis: 能见度（英里）\nwc: 风寒指数\nwdir: 风向（角度）\nwdir_cardinal: 风向的助记符号\ngust: 风速瞬时值（英里/小时）\nwspd: 风速（英里/小时）\nmax_temp: 最高温度（华氏度）\nmin_temp: 最低温度（华氏度）\nprecip_total: 总降水量（英寸）\nprecip_hrly: 每小时降水量（英寸）\nsnow_hrly: 每小时降雪量（英寸）\nuv_desc: 紫外线指数描述\nfeels_like: 体感温度（华氏度）\nuv_index: 紫外线指数\nqualifier: 气象数据的限定条件\nqualifier_svrty: 限定条件的严重程度\nblunt_phrase: 概括的天气短语描述\nterse_phrase: 简短的天气短语描述\nclds: 云量\nwater_temp: 水温\nprimary_wave_period: 主波周期\nprimary_wave_height: 主波高度\nprimary_swell_period: 主涌浪周期\nprimary_swell_height: 主涌浪高度\nprimary_swell_direction: 主涌浪方向\nsecondary_swell_period: 次涌浪周期\nsecondary_swell_height: 次涌浪高度\nsecondary_swell_direction: 次涌浪方向'

#### 1. 数据非空率、分布情况（词频率）、异常值比例

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

cities_ls = ['BAY','Chicago','DC','LA','Melbourne','NYC']
suffix = 'Weather_ori_1.csv'
# col = ['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'REPORTTYPE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure', 'HOURLYPressureTendency', 'HOURLYPressureChange', 'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting', 'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather', 'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth', 'DAILYAverageStationPressure', 'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed', 'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed', 'DAILYSustainedWindDirection', 'MonthlyMaximumTemp', 'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH', 'MonthlyDewpointTemp', 'MonthlyWetBulbTemp', 'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays', 'MonthlyStationPressure', 'MonthlySeaLevelPressure', 'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall', 'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp', 'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip', 'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip', 'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall', 'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth', 'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp', 'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp', 'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip', 'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow', 'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate', 'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue', 'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime', 'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays', 'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD', 'MonthlyTotalSeasonToDateHeatingDD', 'MonthlyTotalSeasonToDateCoolingDD']
col = ['key', 'class', 'expire_time_gmt', 'obs_id', 'obs_name', 'valid_time_gmt', 
         'day_ind', 'temp', 'wx_icon', 'icon_extd', 'wx_phrase', 'pressure_tend', 
         'pressure_desc', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 
         'wdir', 'wdir_cardinal', 'gust', 'wspd', 'max_temp', 'min_temp', 
         'precip_total', 'precip_hrly', 'snow_hrly', 'uv_desc', 'feels_like', 
         'uv_index', 'qualifier', 'qualifier_svrty', 'blunt_phrase', 'terse_phrase', 
         'clds', 'water_temp', 'primary_wave_period', 'primary_wave_height', 
         'primary_swell_period', 'primary_swell_height', 'primary_swell_direction', 
         'secondary_swell_period', 'secondary_swell_height', 'secondary_swell_direction']
# 适合分析和画图的列
draw_col = ['temp', 'wx_phrase', 'pressure_tend', 'pressure_desc',\
            'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc',\
            'wdir', 'wdir_cardinal', 'gust', 'wspd', 'max_temp',\
            'min_temp', 'precip_total', 'precip_hrly', 'snow_hrly',\
            'uv_desc', 'feels_like', 'uv_index', 'clds', ]

for c in cities_ls:
    f_str = c + suffix # the name of the file
    df_statistics = pd.read_csv(f_str)
    num_rows = len(df_statistics)
    print(c, "Number of rows:", num_rows)

    # 统计一下数据的非空率（即 1 减去 缺失率 ）
    prop_non_null_df = pd.DataFrame(columns=col)
    for col_name in col:
        prop_non_null = df_statistics[col_name].count() / num_rows
        prop_non_null_df.loc[0, col_name] = prop_non_null
        
    prop_non_null_path = f'./statistics/{c}/'
    prop_non_null_file = c+'Weather_ori_1_prop_non_null.csv'
    if not os.path.exists(prop_non_null_path):
        os.makedirs(prop_non_null_path)
    prop_non_null_df.to_csv(prop_non_null_path+prop_non_null_file,\
                          index=False)
    
    # 统计一下分布情况
    for col_name in draw_col:
        distribution = df_statistics[col_name].value_counts() / num_rows
        if len(distribution) == 0: continue
        distribution_path = f'./statistics/{c}/distribution/'
        if not os.path.exists(distribution_path):
            os.makedirs(distribution_path)
        distribution_file = c+f'Weather_ori_1_{col_name}.csv'
        distribution.to_csv(distribution_path+distribution_file)
        
        distribution.sort_index(inplace=True) # 先根据索引值排序
        # 区域图
        distribution.plot.area(x=distribution.index, stacked=False)

        # 折线图
        # distribution.plot.line()

        # 柱状图
        # distribution.plot.bar()

        plt.grid(color = 'r', linestyle = '--', linewidth = 0.5)
        if col_name == 'wx_phrase': plt.xticks(rotation=15)
        plt.savefig((distribution_path+distribution_file)[:-4], dpi=300)
        plt.close()


BAY Number of rows: 4849
Chicago Number of rows: 43734
DC Number of rows: 47368
LA Number of rows: 3197
Melbourne Number of rows: 240747
NYC Number of rows: 43431


#### 2. 异常值

In [15]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import Workbook

cities_ls = ['BAY','Chicago','DC','LA','Melbourne','NYC']
suffix = 'Weather_ori_1.csv'
# col = ['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'REPORTTYPE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure', 'HOURLYPressureTendency', 'HOURLYPressureChange', 'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting', 'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather', 'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth', 'DAILYAverageStationPressure', 'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed', 'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed', 'DAILYSustainedWindDirection', 'MonthlyMaximumTemp', 'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH', 'MonthlyDewpointTemp', 'MonthlyWetBulbTemp', 'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays', 'MonthlyStationPressure', 'MonthlySeaLevelPressure', 'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall', 'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp', 'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip', 'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip', 'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall', 'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth', 'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp', 'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp', 'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip', 'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow', 'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate', 'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue', 'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime', 'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays', 'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD', 'MonthlyTotalSeasonToDateHeatingDD', 'MonthlyTotalSeasonToDateCoolingDD']
col = ['key', 'class', 'expire_time_gmt', 'obs_id', 'obs_name', 'valid_time_gmt', 
         'day_ind', 'temp', 'wx_icon', 'icon_extd', 'wx_phrase', 'pressure_tend', 
         'pressure_desc', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 
         'wdir', 'wdir_cardinal', 'gust', 'wspd', 'max_temp', 'min_temp', 
         'precip_total', 'precip_hrly', 'snow_hrly', 'uv_desc', 'feels_like', 
         'uv_index', 'qualifier', 'qualifier_svrty', 'blunt_phrase', 'terse_phrase', 
         'clds', 'water_temp', 'primary_wave_period', 'primary_wave_height', 
         'primary_swell_period', 'primary_swell_height', 'primary_swell_direction', 
         'secondary_swell_period', 'secondary_swell_height', 'secondary_swell_direction']
# 适合分析和画图的列
draw_col = ['temp', 'wx_phrase', 'pressure_tend', 'pressure_desc',\
            'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc',\
            'wdir', 'wdir_cardinal', 'gust', 'wspd', 'max_temp',\
            'min_temp', 'precip_total', 'precip_hrly', 'snow_hrly',\
            'uv_desc', 'feels_like', 'uv_index', 'clds', ]


for c in cities_ls:
    f_str = c + suffix # the name of the file
    df_statistics = pd.read_csv(f_str)
    num_rows = len(df_statistics)
    print(c, "Number of rows:", num_rows)

    outlier_ratio_df = pd.DataFrame(columns=col) # 异常值比例
    # 以数据为主的列，以及可能出现异常值的列
    number_col = ['temp', 'pressure_tend', 'dewPt', \
                    'heat_index', 'rh', 'pressure', 'vis', 'wc',\
                    'wdir', 'gust', 'wspd', 'max_temp',\
                    'min_temp', 'precip_total', 'precip_hrly', 'snow_hrly',\
                    'feels_like', 'uv_index', ]
    df_2_list = [] # 超过2标准差的异常值列表
    df_3_list = [] # 超过3标准差的异常值列表

    for col_name in number_col:
        # 1. 统计异常值（大于两个标准差）比例
        data = df_statistics[col_name]
        if len(data) == 0: continue

        mean = data.mean()
        std = data.std()

        mask_2 = (data-mean).abs() > 2 * std
        mask_3 = (data-mean).abs() > 3 * std

        outlier_ratio_2 = mask_2.mean()
        outlier_ratio_3 = mask_3.mean()
        outlier_ratio_df.loc['std', col_name] = std
        outlier_ratio_df.loc['ratio_out_std_2', col_name] = outlier_ratio_2
        outlier_ratio_df.loc['ratio_out_std_3', col_name] = outlier_ratio_3

        # 2. 查找异常值
        dict_find_2 = {'valid_time_gmt':[], col_name:[]} # 两个标准差
        dict_find_3 = {'valid_time_gmt':[], col_name:[]} # 三个标准差
        data_find = df_statistics.loc[:, ['valid_time_gmt', col_name]]
        for i in range(len(mask_2)):
            if mask_2[i] == 1:
                dict_find_2['valid_time_gmt'].append(data_find.iloc[i, 0])
                dict_find_2[col_name].append(data_find.iloc[i, 1])
            if mask_3[i] == 1:
                dict_find_3['valid_time_gmt'].append(data_find.iloc[i, 0])
                dict_find_3[col_name].append(data_find.iloc[i, 1])
        outlier_value_2_df = pd.DataFrame(dict_find_2)
        outlier_value_3_df = pd.DataFrame(dict_find_3)
        df_2_list.append(outlier_value_2_df)
        df_3_list.append(outlier_value_3_df)

        outlier_value_path_2 = f'./statistics/{c}/outlier_value_2std/'  
        outlier_value_path_3 = f'./statistics/{c}/outlier_value_3std/'
        outlier_value_file = c+f'Weather_ori_1_{col_name}_outlier_value.csv'
        
        if not os.path.exists(outlier_value_path_2):
            os.makedirs(outlier_value_path_2)
        if not os.path.exists(outlier_value_path_3):
            os.makedirs(outlier_value_path_3)
        outlier_value_2_df.to_csv(outlier_value_path_2+outlier_value_file,\
                          index=False)
        outlier_value_3_df.to_csv(outlier_value_path_3+outlier_value_file,\
                          index=False)
    
    wb_2 = Workbook()
    wb_2.save(outlier_value_path_2+c+f'_ALL_Weather_ori_1_outlier_value.xlsx')
    wb_3 = Workbook()
    wb_3.save(outlier_value_path_3+c+f'_ALL_Weather_ori_1_outlier_value.xlsx')
    for df, col_name in zip(df_2_list,number_col):
        with pd.ExcelWriter(outlier_value_path_2+c+f'_ALL_Weather_ori_1_outlier_value.xlsx', engine='openpyxl', mode='a') as writer:
            writer.book = wb_2
            df.to_excel(writer, sheet_name=col_name)
    for df, col_name in zip(df_3_list,number_col):
        with pd.ExcelWriter(outlier_value_path_3+c+f'_ALL_Weather_ori_1_outlier_value.xlsx', engine='openpyxl', mode='a') as writer:
            writer.book = wb_3
            df.to_excel(writer, sheet_name=col_name)

    # writer_2 = pd.ExcelWriter(outlier_value_path_2+c+f'_ALL_Weather_ori_1_outlier_value.xlsx' ,mode='a',engine='openpyxl')
    # writer_3 = pd.ExcelWriter(outlier_value_path_3+c+f'_ALL_Weather_ori_1_outlier_value.xlsx' ,mode='a',engine='openpyxl')
    # for df, col_name in zip(df_2_list,number_col):
    #     df.to_excel(writer_2, col_name)
    # for df, col_name in zip(df_3_list,number_col):
    #     df.to_excel(writer_3, col_name)
    # writer_2.save()
    # writer_2.close()
    # writer_3.save()
    # writer_3.close()

    outlier_ratio_path = f'./statistics/{c}/'
    outlier_ratio_file = c+'Weather_ori_1_outlier_ratio.csv'
    if not os.path.exists(outlier_ratio_path):
        os.makedirs(outlier_ratio_path)
    outlier_ratio_df.to_csv(outlier_ratio_path+outlier_ratio_file,\
                          index=True)

outlier_ratio_df

BAY Number of rows: 4849


KeyError: "Worksheet 'Sheet' does not exist."

#### 3. 先对其进行处理，使每小时内的所有数据取平均值
（1）对于非数字的列，取第一次出现的  
（2）对于数字的列，若非‘最高气温’和‘最低气温’这两列，则取平均值  
（3）对于最高（最低）气温，取最大（最小）值  
（4）把expire_time替换成修改前的valid_time，再由（1），则处理后的expire_time，即为每小时的第一条数据的具体时间  

In [7]:
import os
import pandas as pd
from tqdm import tqdm

cities_ls = ['NYC', 'BAY','Chicago','DC','LA','Melbourne']
suffix = 'Weather_ori_1.csv'
# col = ['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'REPORTTYPE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure', 'HOURLYPressureTendency', 'HOURLYPressureChange', 'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting', 'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather', 'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth', 'DAILYAverageStationPressure', 'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed', 'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed', 'DAILYSustainedWindDirection', 'MonthlyMaximumTemp', 'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH', 'MonthlyDewpointTemp', 'MonthlyWetBulbTemp', 'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays', 'MonthlyStationPressure', 'MonthlySeaLevelPressure', 'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall', 'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp', 'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip', 'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip', 'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall', 'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth', 'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp', 'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp', 'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip', 'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow', 'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate', 'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue', 'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime', 'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays', 'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD', 'MonthlyTotalSeasonToDateHeatingDD', 'MonthlyTotalSeasonToDateCoolingDD']
col = ['key', 'class', 'expire_time_gmt', 'obs_id', 'obs_name', 'valid_time_gmt', 
         'day_ind', 'temp', 'wx_icon', 'icon_extd', 'wx_phrase', 'pressure_tend', 
         'pressure_desc', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 
         'wdir', 'wdir_cardinal', 'gust', 'wspd', 'max_temp', 'min_temp', 
         'precip_total', 'precip_hrly', 'snow_hrly', 'uv_desc', 'feels_like', 
         'uv_index', 'qualifier', 'qualifier_svrty', 'blunt_phrase', 'terse_phrase', 
         'clds', 'water_temp', 'primary_wave_period', 'primary_wave_height', 
         'primary_swell_period', 'primary_swell_height', 'primary_swell_direction', 
         'secondary_swell_period', 'secondary_swell_height', 'secondary_swell_direction']
number_col = ['temp', 'pressure_tend', 'dewPt', 'heat_index', 'rh',\
              'pressure', 'vis', 'wc', 'gust',\
              'wspd', 'precip_total', 'precip_hrly', 'snow_hrly',\
              'feels_like', 'uv_index']
special_col = ['max_temp','min_temp'] # TODO
for c in cities_ls:
    f_str = c + suffix # the name of the file
    df_process = pd.read_csv(f_str)
    num_rows = len(df_process)
    print(c, "Number of rows:", num_rows)

    df_process['expire_time_gmt'] = df_process['valid_time_gmt'].copy()
    df_process['valid_time_gmt'] = df_process['valid_time_gmt'].apply(lambda x: str(x)[:-6])
    

    new_dict = {}
    for col_name in col:
        new_dict[col_name] = []
    last_date = 0
    tmp_dict = {}
    same_cnt = 0

    for i in tqdm(range(num_rows)):
        now_date = df_process.loc[i, 'valid_time_gmt']
        # DATE不相同，需要处理，其中前后边界需要特别处理
        if last_date != now_date:
            # 先把数据的列除以相同的行数，即转换为平均值
            for col_name in number_col:
                if i != 0: tmp_dict[col_name] /= same_cnt
            # 取最大or最小值
            if i != 0: tmp_dict['max_temp'] = max(tmp_dict['max_temp'])
            if i != 0: tmp_dict['min_temp'] = min(tmp_dict['min_temp'])
            # 然后把tmp_dict插入new_dict中
            for col_name in col:
                if i != 0 or i == num_rows-1: 
                    new_dict[col_name].append(tmp_dict[col_name])
            # 并初始化新的tmp_dict
            for col_name in col:
                tmp_dict[col_name] = df_process.loc[i, col_name]
            for col_name in number_col:
                tmp_dict[col_name] = float(tmp_dict[col_name])
            for col_name in special_col:
                tmp_dict[col_name] = [float(tmp_dict[col_name])]
                

            # 开始新的一轮相同DATE的行的处理，初始化↓
            same_cnt = 1
            last_date = now_date

        # DATE相同，加入到tmp_dict中
        else:
            # 计数加1
            same_cnt += 1
            # 直接加到tmo_dict（求和），最后再进行求平均值
            # 对于不在number_col中的列，其为字符串无法取平均值，只保留第一个的数据，即上方初始化的值
            for col_name in number_col:
                tmp_dict[col_name] += float(df_process.loc[i, col_name])
            # 对于最大值or最小值，先加入list中，后续再处理
            for col_name in special_col:
                tmp_dict[col_name].append(float(df_process.loc[i, col_name]))

            # 特殊处理尾端
            if i == num_rows-1:
                # 先转化为平均值
                for col_name in number_col:
                    tmp_dict[col_name] /= same_cnt
                # 取最大or最小值
                tmp_dict['max_temp'] = max(tmp_dict['max_temp'])
                tmp_dict['min_temp'] = min(tmp_dict['min_temp'])
                # 然后把tmp_dict插入new_dict中
                for col_name in col:
                    new_dict[col_name].append(tmp_dict[col_name])

    new_df = pd.DataFrame(new_dict)
    process_path = f'./process/{c}/'
    process_file = c+'Weather_1h.csv'
    if not os.path.exists(process_path):
        os.makedirs(process_path)
    new_df.to_csv(process_path+process_file, index=False)

   

NYC Number of rows: 43431


 17%|█▋        | 806/4849 [00:00<00:00, 4084.32it/s]

BAY Number of rows: 4849


  1%|          | 375/43734 [00:00<00:11, 3749.18it/s]

Chicago Number of rows: 43734


...

  0%|          | 0/3197 [00:00<?, ?it/s]

LA Number of rows: 3197


  0%|          | 517/240747 [00:00<00:46, 5168.96it/s]

Melbourne Number of rows: 240747


100%|██████████| 240747/240747 [00:43<00:00, 5508.19it/s]


#### 3. 按照train/test的视角去分析